## Exercise using SimpliVity Powershell Module

Although it is fun to use the REST API primitive and issue them one-step-a-time, it's not a very productive in helping you manage your datacenter. This is because many use cases will involve other datacenter components such as VMware vCenter, Microsoft domain Controller, Networking gears etc.
To increase your agility in managing SimpliVity HCI, we are going to show you how to use both the SimpliVity and VMware PowerShell module. This combination allows the admin to perform many repetitive task, providing additional opportunities to enhance your business.

**Use case** We are planning to deploy a VM by cloning it from a known template, powering it up and waiting until the OS is up and running.

### Let's see how we can automate this use case by using SimpliVity PowerShell module and VMware PowerCLI 
<div class="alert alert-block alert-warning">
<b>READ FIRST!</b> Please wait until the modules are loaded and displayed in your notebook.
</div>


In [ ]:
# The SimpliVity module has been allocated to the /user/local/share/PowerShell/Modules folder
try {
    Get-Module -ListAvailable | where-Object Name -match HPESimplivity
} catch {
    install-module HPESimplivity
} 
# Load the PowerCLI and adjust the configuration accordingly,this can take about 1 minute, please wait until you see response.
try {
    Get-Module -ListAvailable | where Name -match VMware
    
} catch {
    Install-Module -Name VMware.PowerCLI
    Get-Module -ListAvailable | where Name -match VMware
}
Set-PowerCLIConfiguration -Scope User -ParticipateInCEIP $true -Confirm:$false | format-table
Set-PowerCLIConfiguration -Scope User -InvalidCertificateAction Ignore -Confirm:$false | format-table

## Entering credential for SimpliVity Controller

Username is your login in format of **root**  (which is assigned at the beginning of this lab)<br>

Password is your password provided for Discover Virtual Event 2020 which is **vmware** <br>

In [ ]:
# Create credential for VMware vCenter that is related to the SimpliVity
$hms = "16.31.86.185"
$hms_username = "root"
$hms_password = "vmware"
$mva_ip = "16.31.87.101"
$secPasswd = ConvertTo-SecureString $hms_password -AsPlainText -Force
$cred = New-Object System.Management.Automation.PSCredential ($hms_username, $secPasswd)
$cred

### Create both sessions to SimpliVity and vCenter and obtain the required Tokens

In [ ]:
# Obtain SimpliVity token using the credential provided and SimpliVity module will handle token expiration

$SVTtoken = Connect-SVT -OVC $mva_ip -Credential $cred
$SVTtoken

In [ ]:
# Connect to this webinar federation vCenter 
$hmsToken = connect-VIserver -Server $hms -Cred $cred -Force
$hmsToken

### SimpliVity federation content from HPESimpliVity module using the Get-SVTvm cmdlet

In [ ]:
Get-SVTvm | format-table -Property VMname,ClusterName,DataCenterName -autosize

### SimpliVity federation content from the VMware.PowerCLI module using Get-VM cmdlet. 
<div class="alert alert-block alert-success">
<b>READ FIRST!</b> Using the PowerCLI Get-VM commandlet, you can display the list of the VM and the associated clustername and datacenter name including each of the OVC.
</div>

In [ ]:
Get-VM | Sort-Object -Property Name | format-table -Property Name,@{Name=’ClusterName’;Expression={$_.VMHost.Parent}},@{Name='DatacenterName';Expression={$_.VMHost.parent.parentfolder.parent}}

### Let's discover a VM named Ubuntu-vdbench-* located at the DataCenter CCP_Virt_0000 

In [ ]:
$VM = Get-SVTvm -Name 'Ubuntu-vdbench-*' | where-Object DataCenterName -match CCP_Virt_0000 | select-Object -First 1
$VM 

### I am using the current time to create the cloned VM name

In [ ]:
$NewVMname = 'My_VM_' + (Get-Date -Format FileDateTime).ToString()
$NewVMname

### Let's clone the template to the VM with the new name
- This operation can take up to 20 seconds
- Please select the LasVegas vCenter and observed the clone process and validate that the VM cloned succesfully

In [ ]:
$Status = New-SVTclone -VmName $VM.VMname -CloneName $NewVMname
$Status
Get-SVTtask -ID $Status.TaskID

### Let's power up the newly cloned VM and wait until the VM state is *PoweredON*

In [ ]:
$vmup = Get-VM -Name $NewVMname
if ($vmup.PowerState -eq "PoweredOff") {
    Write-Output "Demo VM $vmup is Switching ON"
    Start-VM -VM $vmup -Confirm:$false
    $vmup | Get-VMQuestion | Set-VMQuestion -Option "I copied it" -Confirm:$false
}
$vmup

### But hold on, the OS is not completely booted-up yet. We are using the VMware *toolStatus* to ensure that it's really ready

In [ ]:
Do { 
    $toolsStatus = (Get-VM -Name $NewVMname).extensiondata.Guest.ToolsStatus
    write-Output $toolsStatus
    start-sleep -Seconds 1
} until ($toolsStatus -ne "toolsNotRunning")

### Conclusion: Combining multiple modules for different components is key to agility!

In [ ]:
Get-VM -Name $NewVMname | format-table -autosize

# Continue now to Lab 3: Python Basic Example
Description: In this intro we will be using Python to connect to the REST API endpoint and retrieve an authentication token to perform another exercise.

* [Lab 3](PythonBasicExample.ipynb)

# Or return back to the Introduction Notebook
Description: This will allow you to try other notebooks.

* [Introduction](Introduction.ipynb)